# Testing Rome2Rio API

Goal of this notebook, calculate co2 emission and offset costs. Example site: https://co2.myclimate.org/en/offset_further_emissions


### Search

Example request and explanation of resposne in [docs](https://www.rome2rio.com/documentation/1-4/search/)

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests

load_dotenv()
ROME2RIO_KEY = os.getenv("ROME2RIO_KEY")

S = requests.Session()

In [ ]:
Origin = "Bern"
Destination = "Zurich"

URL = "http://free.rome2rio.com/api/1.4/json/Search"

PARAMS = {
    "key": ROME2RIO_KEY,
    "oName": Origin,
    "dName": Destination,
    "noRideshare": True,
    "currencyCode": "EUR"
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

In [ ]:
# print(DATA)

Print available routes, transit time and estimated costs

In [ ]:
fields = ['name', 'distance', 'totalTransitDuration', 'totalDuration', 'indicativePrices']
routes = [
    {k: route[k] for k in fields}
    for route in DATA['routes']
]

In [ ]:
# routes

### Now calculate CO2 emmission.

Need some baseline metrics for different transport modes. Taken from this [bbc blog](https://www.bbc.com/news/science-environment-493495660)

In [ ]:
emissions_pp = {
    "flight_domestic": 254,
    "flight_longhaul": 195,
    "car_1pax": 171,
    "bus": 104,
    "car_4pax": 43,
    "rail": 41,
    "coach": 27,
    "eurostar": 6,
}

Aggregating it to the Rome2Rio categories we have:

In [ ]:
from statistics import mean 

emissions_pp_r2r = {
    "Drive": mean([emissions_pp['car_1pax'], emissions_pp['car_4pax']]),
    "Bus": mean([emissions_pp['bus'], emissions_pp['coach']]),
    "Train": mean([emissions_pp['rail'], emissions_pp['eurostar']]),
}

So for the above 3 routes, the CO2 emission (in grams) for each option is:

In [ ]:
# add co2 for each route
for route in routes:
    route.update(
        {"co2Emission": emissions_pp_r2r[route['name']] * route['distance']}
    )

In [ ]:
[
    {k: route[k] for k in ['name', 'co2Emission']}
    for route in routes
]

Now translate into how many trees you will need to compensate a gram of CO2. 

Using the calculation they make on [Trees for the Future](https://www.tfaforms.com/4666774)

> Trees for the Future's agroforestry projects in Sub-Saharan Africa sequester CO2 through tree planting and land restoration at a cost of USD 6.91 per metric tonne of CO2. Therefore, the total cost to offset your annual CO2 emissions is: $ 0.00

So we need to convert the grams to metric tonnes of CO2 first: 1 tonne is 1_000_000 grams.

In [ ]:
# conver to EUR
offset_costs = 6.91 * 0.82

# add co2 compensation costs for each route
for route in routes:
    route.update(
        {"co2CompensationCost": round(route["co2Emission"] / 1_000_000 * offset_costs, 2)}
    )

[
    {k: route[k] for k in ['name', 'co2CompensationCost']}
    for route in routes
]

Note: this only gives us an estimate of the cost to compensate. It doesn't tell us how many trees will be planted.

Estimates vary on how much CO2 one tree will offset:
- 0.2 tonne per tree. [This calculator](http://www.carbonify.com/carbon-calculator.htm) claims that 5 trees will need to be planted to ensure that at least one lives 40 years equalling 1 tonne of CO2 absorbtion. 
- 1 tonne per 100 years according to [here](https://www.viessmann.co.uk/heating-advice/how-much-co2-does-tree-absorb#:~:text=Over%20a%20lifetime%20of%20100,to%20offset%20the%20emissions%20created.)
- 7 tonnes according to [this site](https://ythakker.medium.com/how-many-new-trees-would-it-take-to-offset-our-carbon-emissions-13c78ccb09ab)


In [ ]:
# add nr of trees to be planted
for route in routes:
    route.update(
        {"co2CompensationTrees": round(route["co2Emission"] / 1_000_000 * 5, 2)}
    )

[
    {k: route[k] for k in ['name', 'co2CompensationTrees']}
    for route in routes
]

In [ ]:
print(f"Details for trip {Origin} to {Destination}:")
(
    pd.DataFrame(routes)
    [['name', 'totalDuration', 'co2Emission', 'co2CompensationCost', 'co2CompensationTrees']]
    .rename({
        "totalDuration": "totalDuration (min)",
        "co2Emission": "co2Emission (grams)",
        "co2CompensationCost": "co2CompensationCost (EUR)",
        "co2CompensationTrees": "co2CompensationTrees (# Trees)"}, axis=1)
)

Done.

TODO:
- Names are not pure: name is based on segments. So need to build up CO2 based on segments instead of the pure name.
    - See for example: Amsterdam -> Paris
- Add other transport modes: i.e. flight? Boat? Mixed?